In [1]:
import numpy as np
import math
from dataclasses import dataclass
from typing import Callable, Dict, Optional

np.seterr(all="raise")  # Error on overflow


@dataclass
class FoundationParameters:
    REO_DENSITY: float = 7850  # kg/m3
    LOAD_DEAD: float = 8000  # kN
    LOAD_LIVE: float = 2500  # kN
    BEARINGPRESSURE: float = 500  # kPa
    FTG_COVER: float = 0.060  # m
    COLUMN_WIDTH: float = 0.500  # m


@dataclass
class Calculation:
    func: Callable
    short_name: str
    long_name: str
    definition: str = ""
    format_func: Optional[Callable[[float], str]] = None

    def format_value(self, value: float) -> str:
        if self.format_func:
            return self.format_func(value)
        elif isinstance(value, float):
            return f"{value:.3f}"
        else:
            return str(value)


class FoundationCalculator:
    def __init__(self, params: FoundationParameters):
        self.params = params
        self.calculations: Dict[str, Calculation] = {}

    def add_calculation(self, name: str, calc: Calculation):
        self.calculations[name] = calc

    def calculate(self, sizes: np.ndarray) -> np.ndarray:
        results = {name: sizes[name] for name in sizes.dtype.names}
        calculated = set(sizes.dtype.names)

        for name, calc in self.calculations.items():
            args = calc.func.__code__.co_varnames
            missing_args = [arg for arg in args if arg not in results]

            if not missing_args:
                results[name] = calc.func(**{arg: results[arg] for arg in args})
                calculated.add(name)
            else:
                print(f"Skipping {name}. Missing arguments: {', '.join(missing_args)}")

        uncalculated = set(self.calculations.keys()) - calculated
        if len(uncalculated) > 0:
            raise ValueError(
                f"Unable to calculate all fields. Uncalculated fields: {', '.join(uncalculated)}"
            )

        new_dtype = sizes.dtype.descr + [
            (name, "f8") for name in results if name not in sizes.dtype.names
        ]
        new_sizes = np.empty(sizes.shape, dtype=new_dtype)
        for name in sizes.dtype.names:
            new_sizes[name] = sizes[name]
        for name, result in results.items():
            if name not in sizes.dtype.names:
                new_sizes[name] = result

        return new_sizes


class FoundationSizer:
    def __init__(self, params: FoundationParameters, calculator: FoundationCalculator):
        self.params = params
        self.calculator = calculator
        self.sizes = self.generate_foundation_sizes()
        self.calculate_all_properties()
        self.column_names = list(self.sizes.dtype.names)

    def generate_foundation_sizes(self) -> np.ndarray:
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt(
                    (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    / self.params.BEARINGPRESSURE
                )
                / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * self.params.COLUMN_WIDTH**2) / 3
                        + (1 + (3 * self.params.BEARINGPRESSURE) / 19040)
                        * (self.params.LOAD_DEAD + self.params.LOAD_LIVE)
                    )
                    - (9520 + 3 * self.params.BEARINGPRESSURE)
                    * self.params.COLUMN_WIDTH
                )
                / (19040 + 3 * self.params.BEARINGPRESSURE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        return np.rec.array(
            foundation_sizes,
            dtype=[
                ("FtgLength", "f4"),
                ("FtgDepth", "f4"),
                ("fc", "f4"),
                ("ReoSize", "f4"),
                ("ReoCts", "f4"),
            ],
        )

    def calculate_all_properties(self) -> None:
        self.sizes = self.calculator.calculate(self.sizes)

    def remove_fails(self) -> None:
        valid_mask = (
            (self.sizes["Mrat"] <= 1)
            & (self.sizes["VLrat"] <= 1)
            & (self.sizes["VPrat"] <= 1)
            & (self.sizes["Astact"] >= self.sizes["Astreq"])
            & (self.sizes["Astact"] >= self.sizes["AstreqW"])
            & (self.sizes["BPrat"] <= 1)
        )
        self.sizes = self.sizes[valid_mask]

    def filter_match(self, **kwargs) -> None:
        mask = np.ones(len(self.sizes), dtype=bool)
        for key, value in kwargs.items():
            if key in self.column_names:
                mask &= self.sizes[key] == value
        self.sizes = self.sizes[mask]

    def sort_by_cost(self) -> None:
        self.sizes = np.sort(self.sizes, order="Cost")

    def print_foundation_details(self, row_index: int) -> None:
        if row_index < 0 or row_index >= len(self.sizes):
            print(f"Row index {row_index} is out of range.")
            return

        row = self.sizes[row_index]
        max_name_length = max(
            len(calc.long_name) for calc in self.calculator.calculations.values()
        )
        column_width = max_name_length + 15  # Add some extra space for the short name

        print(f"Details for foundation size at row {row_index}:")

        # Print the foundation properties first
        properties = [
            ("Foundation Length", "L", "FtgLength", lambda x: f"{x*1000:.0f} mm"),
            ("Foundation Depth", "D", "FtgDepth", lambda x: f"{x*1000:.0f} mm"),
            ("Concrete Strength", "fc", "fc", lambda x: f"{x:.0f} MPa"),
            ("Reinforcement Size", "Bar", "ReoSize", lambda x: f"N{x*1000:.0f}"),
            ("Reinforcement Spacing", "CTS", "ReoCts", lambda x: f"{x*1000:.0f} mm"),
        ]

        for long_name, short_name, field_name, format_func in properties:
            value = row[field_name]
            formatted_value = format_func(value)
            print(
                "{:<{width}}  {}".format(
                    f"{long_name} ({short_name}):", formatted_value, width=column_width
                )
            )

        # Print the calculations
        for name, calc in self.calculator.calculations.items():
            value = row[name]
            formatted_value = calc.format_value(value)
            print(
                "{:<{width}}  {}".format(
                    f"{calc.long_name} ({calc.short_name}):",
                    formatted_value,
                    width=column_width,
                )
            )

    def print_array(self, num_rows: int) -> None:
        if num_rows <= 0:
            print("Number of rows must be greater than 0.")
            return

        if num_rows > len(self.sizes):
            print(f"Only {len(self.sizes)} rows available. Printing all rows.")
            num_rows = len(self.sizes)

        # Determine which fields to display
        display_fields = [
            "FtgLength",
            "FtgDepth",
            "fc",
            "ReoSize",
            "ReoCts",
            "BPrat",
            "Mrat",
            "VPrat",
            "VLrat",
            "Cost",
        ]

        # Create a list of tuples with formatted values
        formatted_data = []
        for row in self.sizes[:num_rows]:
            formatted_row = []
            for field in display_fields:
                value = row[field]
                if field in self.calculator.calculations:
                    calc = self.calculator.calculations[field]
                    formatted_value = calc.format_value(value)
                elif field in ["FtgLength", "FtgDepth"]:
                    formatted_value = f"{value*1000:.0f} mm"
                elif field == "fc":
                    formatted_value = f"{value:.0f} MPa"
                elif field == "ReoSize":
                    formatted_value = f"N{value*1000:.0f}"
                elif field == "ReoCts":
                    formatted_value = f"{value*1000:.0f} mm"
                else:
                    formatted_value = f"{value:.3f}"
                formatted_row.append(formatted_value)
            formatted_data.append(tuple(formatted_row))

        # Create a new structured array with formatted data
        display_array = np.array(
            formatted_data, dtype=[(field, "U20") for field in display_fields]
        )

        # Print the array
        print(f"First {num_rows} rows of the foundation array:")

        # Print column headers
        header = " | ".join(f"{field:^15}" for field in display_fields)
        print(header)
        print("-" * len(header))

        # Print rows
        for row in display_array:
            print(" | ".join(f"{value:^15}" for value in row))


# Usage
params = FoundationParameters()
calculator = FoundationCalculator(params)

# Add calculations
calculator.add_calculation(
    "SWt",
    Calculation(
        func=lambda FtgDepth, FtgLength: (6 * FtgDepth * FtgLength**2),
        short_name="SWt",
        long_name="Self Weight of Footing",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "Pult",
    Calculation(
        func=lambda SWt: (1.2 * (SWt + params.LOAD_DEAD) + 1.5 * params.LOAD_LIVE),
        short_name="P*",
        long_name="Ultimate Load",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "BPmax",
    Calculation(
        func=lambda FtgDepth, FtgLength: (
            6 * FtgDepth * FtgLength**2 + params.LOAD_LIVE + params.LOAD_DEAD
        )
        / (FtgLength**2),
        short_name="BPm",
        long_name="Maximum Bearing Pressure",
        format_func=lambda x: f"{x:.0f} kPa",
    ),
)

calculator.add_calculation(
    "BPrat",
    Calculation(
        func=lambda BPmax: BPmax / params.BEARINGPRESSURE,
        short_name="BPρ",
        long_name="Bearing Pressure Ratio",
        format_func=lambda x: f"{x*100:.1f}%",
    ),
)

calculator.add_calculation(
    "BPult",
    Calculation(
        func=lambda Pult, FtgLength: Pult / FtgLength**2,
        short_name="BP*",
        long_name="Ultimate Bearing Pressure",
        format_func=lambda x: f"{x:.0f} kPa",
    ),
)

calculator.add_calculation(
    "Astact",
    Calculation(
        func=lambda ReoSize, ReoCts: (250000 * ReoSize**2 * np.pi) / ReoCts,
        short_name="Ast",
        long_name="Actual Steel Area",
        format_func=lambda x: f"{x:.0f} mm²",
    ),
)

calculator.add_calculation(
    "Dom",
    Calculation(
        func=lambda FtgDepth, ReoSize: FtgDepth - params.FTG_COVER - ReoSize / 2,
        short_name="ds",
        long_name="Effective Depth",
        format_func=lambda x: f"{x*1000:.0f} mm",
    ),
)


# TO BE IMPLEMENT THE DIFFERENT X & Y CALCS
# def calculate_min_steel_area(FtgDepth, fc, effective_depth):
#     return (228 * FtgDepth**2 * np.sqrt(fc)) / effective_depth

# # Update the original Astmin calculation
# calculator.add_calculation("Astmin_Y", Calculation(
#     func=lambda FtgDepth, fc, Dom_Y: calculate_min_steel_area(FtgDepth, fc, Dom_Y),
#     short_name="Ast min Y",
#     long_name="Minimum Steel Area Y",
#     format_func=lambda x: f"{x:.0f} mm²"
# ))

# # Add the new Astmin_X calculation
# calculator.add_calculation("Astmin_X", Calculation(
#     func=lambda FtgDepth, fc, Dom_X: calculate_min_steel_area(FtgDepth, fc, Dom_X),
#     short_name="Ast min X",
#     long_name="Minimum Steel Area X",
#     format_func=lambda x: f"{x:.0f} mm²"
# ))

calculator.add_calculation(
    "Astmin",
    Calculation(
        func=lambda FtgDepth, fc, Dom: (228 * FtgDepth**2 * np.sqrt(fc)) / Dom,
        short_name="Ast min",
        long_name="Minimum Steel Area",
        format_func=lambda x: f"{x:.0f} mm²",
    ),
)

calculator.add_calculation(
    "AstminW",
    Calculation(
        func=lambda FtgDepth, fc, Dom, ReoSize: (228 * FtgDepth**2 * np.sqrt(fc))
        / (Dom - ReoSize),
        short_name="Ast min Width",
        long_name="Minimum Steel Area Width",
        format_func=lambda x: f"{x:.0f} mm²",
    ),
)

calculator.add_calculation(
    "alpha",
    Calculation(
        func=lambda fc: 0.85 - 0.0015 * fc,
        short_name="α",
        long_name="Stress Block Factor",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "gamma",
    Calculation(
        func=lambda fc: 0.97 - 0.0025 * fc,
        short_name="γ",
        long_name="Stress Block Depth Factor",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "Mult",
    Calculation(
        func=lambda FtgLength, BPult, SWt: (
            (7 * params.COLUMN_WIDTH - 10 * FtgLength) ** 2
            / (8000 * FtgLength**2)
            * (10 * BPult * FtgLength**2 - 9 * SWt)
        ),
        short_name="M*",
        long_name="Ultimate Moment",
        format_func=lambda x: f"{x:.0f} kNm/m",
    ),
)

calculator.add_calculation(
    "Astshr",
    Calculation(
        func=lambda Mult, Dom: (5 * Mult) / (2 * Dom),
        short_name="Ass",
        long_name="Steel Area for Shear",
        format_func=lambda x: f"{x:.0f} mm²/m",
    ),
)

calculator.add_calculation(
    "AstshrW",
    Calculation(
        func=lambda Mult, Dom, ReoSize: (5 * Mult) / (2 * (Dom - ReoSize)),
        short_name="AssW",
        long_name="Steel Area for Shear Width",
        format_func=lambda x: f"{x:.0f} mm²/m",
    ),
)

calculator.add_calculation(
    "Astreq",
    Calculation(
        func=lambda Astmin, Astshr: np.maximum(Astmin, Astshr),
        short_name="Astreq",
        long_name="Required Steel Area",
        format_func=lambda x: f"{x:.0f} mm²/m",
    ),
)

calculator.add_calculation(
    "AstreqW",
    Calculation(
        func=lambda AstminW, AstshrW: np.maximum(AstminW, AstshrW),
        short_name="AstreqW",
        long_name="Required Steel Area Width",
        format_func=lambda x: f"{x:.0f} mm²/m",
    ),
)

calculator.add_calculation(
    "ku",
    Calculation(
        func=lambda Astreq, alpha, Dom, fc, gamma, FtgLength: Astreq
        / (2000 * alpha * Dom * fc * gamma * FtgLength),
        short_name="ku",
        long_name="Neutral Axis Parameter",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "phi",
    Calculation(
        func=lambda ku: np.minimum(0.85, np.maximum(0.65, 1.24 - 1.08333 * ku)),
        short_name="φ",
        long_name="Capacity Reduction Factor",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "fMuo",
    Calculation(
        func=lambda Astact, Dom, phi, alpha, fc: (Astact * Dom * phi) / 2
        - (Astact**2 * phi) / (8000 * alpha * fc),
        short_name="ØMuo",
        long_name="Moment Capacity",
        format_func=lambda x: f"{x:.1f} kNm/m",
    ),
)

calculator.add_calculation(
    "Mrat",
    Calculation(
        func=lambda Mult, fMuo: Mult / fMuo,
        short_name="Mρ",
        long_name="Moment Ratio",
        format_func=lambda x: f"{x*100:.1f}%",
    ),
)

calculator.add_calculation(
    "CLR",
    Calculation(
        func=lambda BPult, Dom: BPult * (params.COLUMN_WIDTH + Dom) ** 2,
        short_name="CLR",
        long_name="Column Load Reaction",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "VPult",
    Calculation(
        func=lambda Pult, CLR: Pult - CLR,
        short_name="Vp*",
        long_name="Ultimate Punching Shear",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "fVP",
    Calculation(
        func=lambda ReoSize, Dom, fc: (
            952 * (ReoSize - Dom) * (ReoSize - Dom - params.COLUMN_WIDTH) * np.sqrt(fc)
        ),
        short_name="ØVp",
        long_name="Punching Shear Capacity",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "VPrat",
    Calculation(
        func=lambda VPult, fVP: VPult / fVP,
        short_name="VPρ",
        long_name="Punching Shear Ratio",
        format_func=lambda x: f"{x*100:.1f}%",
    ),
)

calculator.add_calculation(
    "dv",
    Calculation(
        func=lambda Dom, FtgDepth: np.maximum(0.9 * Dom, 0.72 * FtgDepth),
        short_name="dv",
        long_name="Effective Shear Depth",
        format_func=lambda x: f"{x*1000:.0f} mm",
    ),
)

calculator.add_calculation(
    "VOult",
    Calculation(
        func=lambda BPult, Dom, FtgLength, SWt, ReoSize: (
            10 * BPult * FtgLength**2 - 9 * SWt
        )
        / (20 * FtgLength**2)
        * (2 * ReoSize - params.COLUMN_WIDTH - 2 * Dom + FtgLength),
        short_name="VO*",
        long_name="One-Way Ultimate Shear",
        format_func=lambda x: f"{x:.1f} kN/m",
    ),
)

calculator.add_calculation(
    "MOult",
    Calculation(
        func=lambda VOult, Dom, FtgLength, ReoSize: VOult
        / 4
        * (2 * ReoSize - params.COLUMN_WIDTH - 2 * Dom + FtgLength),
        short_name="MO*",
        long_name="One-Way Ultimate Moment",
        format_func=lambda x: f"{x:.1f} kNm/m",
    ),
)

calculator.add_calculation(
    "Ex1",
    Calculation(
        func=lambda VOult, dv, MOult, Astact: np.minimum(
            (np.maximum(VOult * dv, MOult) / dv + VOult) / (2 * 200000 * Astact) * 1000,
            3 / 1000,
        ),
        short_name="Ex1",
        long_name="Longitudinal Shear Strain",
        format_func=lambda x: f"{x*1e6:.1f} ×10⁻⁶",
    ),
)

calculator.add_calculation(
    "kv",
    Calculation(
        func=lambda dv, Ex1: 13 / (25 * (1 + dv) * (1 + 1500 * Ex1)),
        short_name="kv",
        long_name="Shear Factor",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "Angle",
    Calculation(
        func=lambda Ex1: 29 + 7000 * Ex1,
        short_name="θv",
        long_name="Angle of inclination",
        format_func=lambda x: f"{x:.1f}°",
    ),
)

calculator.add_calculation(
    "ks",
    Calculation(
        func=lambda FtgDepth: np.maximum(0.5, (10 / 7) * (1 - FtgDepth)),
        short_name="ks",
        long_name="Size Factor",
        format_func=lambda x: f"{x:.3f}",
    ),
)

calculator.add_calculation(
    "fVuc",
    Calculation(
        func=lambda dv, fc, ks, kv: 700 * dv * np.sqrt(fc) * ks * kv,
        short_name="φVuc",
        long_name="Concrete Shear Strength",
        format_func=lambda x: f"{x:.1f} kN",
    ),
)

calculator.add_calculation(
    "VLrat",
    Calculation(
        func=lambda VOult, fVuc: VOult / fVuc,
        short_name="VLρ",
        long_name="One-way Shear Ratio",
        format_func=lambda x: f"{x*100:.1f}%",
    ),
)

calculator.add_calculation(
    "Cost",
    Calculation(
        func=lambda Astact, FtgLength, FtgDepth, fc: (
            Astact / 1000000 * FtgLength * params.REO_DENSITY * 2 * 3400
            + FtgLength**2 * FtgDepth * (130.866 * np.exp(fc * 0.0111) + 45 + 130)
            + 4 * FtgDepth * FtgLength * 180
        ),
        short_name="Cost",
        long_name="Total Cost",
        format_func=lambda x: f"${x:.0f}",
    ),
)

# Initialize FoundationSizer and perform calculations
foundation = FoundationSizer(params, calculator)

# Remove fails, sort by cost, and print results
# print(foundation.sizes.size)
foundation.remove_fails()
foundation.sort_by_cost()
foundation.print_foundation_details(0)
# foundation.print_array(10)

Details for foundation size at row 0:
Foundation Length (L):                     4650 mm
Foundation Depth (D):                      1850 mm
Concrete Strength (fc):                    20 MPa
Reinforcement Size (Bar):                  N28
Reinforcement Spacing (CTS):               300 mm
Self Weight of Footing (SWt):              240.0 kN
Ultimate Load (P*):                        13638.0 kN
Maximum Bearing Pressure (BPm):            497 kPa
Bearing Pressure Ratio (BPρ):              99.3%
Ultimate Bearing Pressure (BP*):           631 kPa
Actual Steel Area (Ast):                   2053 mm²
Effective Depth (ds):                      1776 mm
Minimum Steel Area (Ast min):              1965 mm²
Minimum Steel Area Width (Ast min Width):  1996 mm²
Stress Block Factor (α):                   0.820
Stress Block Depth Factor (γ):             0.920
Ultimate Moment (M*):                      1435 kNm/m
Steel Area for Shear (Ass):                2020 mm²/m
Steel Area for Shear Width (AssW):         